In [3]:
pip install scapy

  Using cached scapy-2.5.0.tar.gz (1.3 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for scapy: filename=scapy-2.5.0-py2.py3-none-any.whl size=1444326 sha256=02a44771c482d1a23adff9cf4610150a6f3cbd341a2e2aa31d7f400db2e1e5a8
  Stored in directory: c:\users\mark8\appdata\local\pip\cache\wheels\62\14\01\370c0063b5c1482143836c55f39233ec898ec2b01c251771cd
Successfully built scapy
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import sys
from scapy.all import *
from scapy.all import Ether, IP, TCP
from scapy.layers.inet import TCP, UDP, IP
from scapy.layers.tls.record import TLS
from scapy.layers.tls.extensions import ServerName
import datetime
import pandas as pd
from decimal import Decimal, ROUND_HALF_UP
import shutil
import time
import threading
import datetime
from scapy.all import sniff

from scapy.all import conf

# scapy 최적화 옵션 설정
conf.ipv6_enabled = False  # IPv6 비활성화
conf.sniff_promisc = False  # Promiscuous 모드 비활성화
conf.use_pcap = True  # pcap 사용 활성화


def get_flow_time_range(packets):
    start_time = packets[0].time
    end_time = packets[-1].time

    for packet in packets:
        if packet.time < start_time:
            start_time = packet.time
        if packet.time > end_time:
            end_time = packet.time

    return start_time, end_time


flows = {}  # 플로우를 저장할 딕셔너리

def map_tls_version(version):
    tls_versions = {
        769: "TLS 1.0",
        770: "TLS 1.1",
        771: "TLS 1.2",
        772: "TLS 1.3"
    }
    return tls_versions.get(version, "Unknown")



def process_packet(packet):
    if TCP in packet:
        if TLS in packet and (packet[TCP].dport == 443 or packet[TCP].sport == 443):
            # TLS 핸드셰이크 정보 추출
            sni = "N/A"  # 기본값으로 SNI를 "N/A"로 설정
            tls_version = "N/A"  # 기본값으로 TLS 버전을 "N/A"로 설정

            if packet[TLS].haslayer(ServerName):
                sni = packet[TLS][ServerName].servername.decode()
            
            if packet[TLS].haslayer(TLS):
                tls_version_number = packet[TLS][TLS].version
                tls_version = map_tls_version(tls_version_number)
                
            # TCP 헤더 정보 추출
            src_ip = packet[IP].src
            src_port = packet[TCP].sport
            dst_ip = packet[IP].dst
            dst_port = packet[TCP].dport
            
            # 플로우 식별을 위한 5-tuple 생성
            flow_key = (src_ip, src_port, dst_ip, dst_port)
            reverse_flow_key = (dst_ip, dst_port, src_ip, src_port)  # 반대 방향 플로우의 5-tuple
            
            # 플로우에 패킷 정보 추가
            
            if flow_key in flows:
                flows[flow_key]['forward_packets'] += 1
                flows[flow_key]['packet_sizes'].append(len(packet))
                flows[flow_key]['packets'].append(packet)
                flows[flow_key]['end_time'] = packet.time

           
            elif reverse_flow_key in flows:
                flows[reverse_flow_key]['backward_packets'] += 1                          
                flows[reverse_flow_key]['packet_sizes'].append(-len(packet))
                flows[reverse_flow_key]['packets'].append(packet)
                flows[reverse_flow_key]['end_time'] = packet.time
            else:
                flows[flow_key] = {
                    'flow_header': f"Flow : {flow_key}",
                    'src_ip': src_ip,
                    'src_port': src_port,
                    'dst_ip': dst_ip,
                    'dst_port': dst_port,
                    'protocol': packet[IP].proto,
                    'start_time': packet.time,
                    'end_time': packet.time,
                    'forward_packets': 1,
                    'backward_packets': 0,
                    'packet_sizes': [len(packet)],
                    'packets': [packet],
                    'sni': sni,  # SNI 정보 추가
                    'tls_version': tls_version  # TLS 버전 정보 추가
                }

def print_flow(file_path):
    
        for flow_key, flow in flows.items():
            f.write(flow['flow_header'] + '\n')
            f.write(f"StartTime : {datetime.fromtimestamp(float(flow['start_time']))}\n")
            f.write(f"SrcIP : {flow['src_ip']}\n")
            f.write(f"SrcPort : {flow['src_port']}\n")
            f.write(f"Prot : {flow['protocol']}\n")            
            f.write(f"DstPort : {flow['dst_port']}\n")
            f.write(f"DstIP : {flow['dst_ip']}\n")
            f.write(f"Flow Duration : {flow['end_time'] - flow['start_time']} seconds\n")
            f.write(f"EndTime : {datetime.fromtimestamp(float(flow['end_time']))}\n")
            f.write(f"SNI : {flow['sni']}\n")            
            packet_sizes = flow['packet_sizes']
            f.write("PSD : ")
            f.write(", ".join(str(size) for size in packet_sizes))
            f.write("\n")
            f.write(f"Version: {flow['tls_version']}\n")

            f.write(f"Total Packet : {flow['forward_packets'] + flow['backward_packets']}\n")
            f.write(f"Forward Packet : {flow['forward_packets']}\n")
            f.write(f"Backward Packet : {flow['backward_packets']}\n")
            f.write("\n\n")



flows = {}  # 플로우를 저장할 딕셔너리

# 패킷 읽기
packets = rdpcap('LIVE1.pcap')


# Ether

In [2]:
packets.show()

0000 Ether / IP / TCP 163.152.219.203:9394 > 163.152.223.142:56253 PA / Raw
0001 Ether / ARP who has 163.152.219.225 says 163.152.207.4 / Padding
0002 Ether / IP / UDP 163.152.207.40:60216 > 239.255.255.250:ws_discovery / Raw
0003 Ether / IP / TCP 110.76.140.197:https > 163.152.219.203:50807 PA / _TLSEncryptedContent
0004 Ether / ARP who has 163.152.225.230 says 163.152.223.4 / Padding
0005 Ether / ARP who has 169.254.162.173 says 163.152.219.7 / Padding
0006 Ether / IP / UDP 163.152.207.40:60216 > 239.255.255.250:ws_discovery / Raw
0007 Ether / IPv6 / UDP fe80::7e32:f105:7c82:bbfd:60217 > ff02::c:ws_discovery / Raw
0008 Ether / IP / TCP 163.152.223.142:56253 > 163.152.219.203:9394 A / Padding
0009 Ether / IP / TCP 163.152.223.142:56253 > 163.152.219.203:9394 PA / Raw
0010 Ether / IP / TCP 163.152.219.203:50807 > 110.76.140.197:https A
0011 Ether / IP / TCP 163.152.223.142:56253 > 163.152.219.203:9394 PA / Raw
0012 Ether / IP / TCP 163.152.219.203:9394 > 163.152.223.142:56253 A
0013 Et

### Ether mac

In [6]:
# 첫 번째 패킷에서 목적지 MAC 주소 추출
#first_packet = packets[0]# 한개 패킷
# 그래서 for문 이용

for packet in packets:
    destination_mac = packet.dst

    # 추출된 목적지 MAC 주소 출력
    print("Destination MAC: ", destination_mac)

Destination MAC:  c4:ca:2b:7d:5a:63
Destination MAC:  ff:ff:ff:ff:ff:ff
Destination MAC:  01:00:5e:7f:ff:fa
Destination MAC:  a8:a1:59:f2:3e:8f
Destination MAC:  ff:ff:ff:ff:ff:ff
Destination MAC:  ff:ff:ff:ff:ff:ff
Destination MAC:  01:00:5e:7f:ff:fa
Destination MAC:  33:33:00:00:00:0c
Destination MAC:  a8:a1:59:f2:3e:8f
Destination MAC:  a8:a1:59:f2:3e:8f
Destination MAC:  c4:ca:2b:7d:5a:63
Destination MAC:  a8:a1:59:f2:3e:8f
Destination MAC:  c4:ca:2b:7d:5a:63
Destination MAC:  a8:a1:59:f2:3e:8f
Destination MAC:  a8:a1:59:f2:3e:8f
Destination MAC:  c4:ca:2b:7d:5a:63
Destination MAC:  c4:ca:2b:7d:5a:63
Destination MAC:  ff:ff:ff:ff:ff:ff
Destination MAC:  ff:ff:ff:ff:ff:ff
Destination MAC:  a8:a1:59:f2:3e:8f
Destination MAC:  a8:a1:59:f2:3e:8f
Destination MAC:  c4:ca:2b:7d:5a:63
Destination MAC:  ff:ff:ff:ff:ff:ff
Destination MAC:  ff:ff:ff:ff:ff:ff
Destination MAC:  c4:ca:2b:7d:5a:63
Destination MAC:  c4:ca:2b:7d:5a:63
Destination MAC:  ff:ff:ff:ff:ff:ff
Destination MAC:  a8:a1:59:f

In [7]:
# 패킷을 순회하며 Ethernet 2 패킷의 목적지 MAC 주소 추출
for packet in packets:
    if Ether in packet: # packet에 [Ether, IP, TCP]
        eth_packet = packet[Ether]
        destination_mac = eth_packet.dst

        # 추출된 목적지 MAC 주소 출력
        print("Destination MAC: ", destination_mac)

Destination MAC:  c4:ca:2b:7d:5a:63
Destination MAC:  ff:ff:ff:ff:ff:ff
Destination MAC:  01:00:5e:7f:ff:fa
Destination MAC:  a8:a1:59:f2:3e:8f
Destination MAC:  ff:ff:ff:ff:ff:ff
Destination MAC:  ff:ff:ff:ff:ff:ff
Destination MAC:  01:00:5e:7f:ff:fa
Destination MAC:  33:33:00:00:00:0c
Destination MAC:  a8:a1:59:f2:3e:8f
Destination MAC:  a8:a1:59:f2:3e:8f
Destination MAC:  c4:ca:2b:7d:5a:63
Destination MAC:  a8:a1:59:f2:3e:8f
Destination MAC:  c4:ca:2b:7d:5a:63
Destination MAC:  a8:a1:59:f2:3e:8f
Destination MAC:  a8:a1:59:f2:3e:8f
Destination MAC:  c4:ca:2b:7d:5a:63
Destination MAC:  c4:ca:2b:7d:5a:63
Destination MAC:  ff:ff:ff:ff:ff:ff
Destination MAC:  ff:ff:ff:ff:ff:ff
Destination MAC:  a8:a1:59:f2:3e:8f
Destination MAC:  a8:a1:59:f2:3e:8f
Destination MAC:  c4:ca:2b:7d:5a:63
Destination MAC:  ff:ff:ff:ff:ff:ff
Destination MAC:  ff:ff:ff:ff:ff:ff
Destination MAC:  c4:ca:2b:7d:5a:63
Destination MAC:  c4:ca:2b:7d:5a:63
Destination MAC:  ff:ff:ff:ff:ff:ff
Destination MAC:  a8:a1:59:f

### Ether src

In [8]:
# 패킷을 순회하며 Ethernet 2 패킷의 src MAC 주소 추출
for packet in packets:
    if Ether in packet:
        eth_packet = packet[Ether]
        source_mac = eth_packet.src

        # 추출된 목적지 MAC 주소 출력
        print("Source MAC: ", source_mac)

Source MAC:  a8:a1:59:f2:3e:8f
Source MAC:  00:1a:f4:f8:a5:93
Source MAC:  24:f5:aa:eb:40:27
Source MAC:  c4:ca:2b:7d:5a:63
Source MAC:  00:1a:f4:f8:a5:30
Source MAC:  00:1a:f4:f8:a5:d9
Source MAC:  24:f5:aa:eb:40:27
Source MAC:  24:f5:aa:eb:40:27
Source MAC:  c4:ca:2b:7d:5a:63
Source MAC:  c4:ca:2b:7d:5a:63
Source MAC:  a8:a1:59:f2:3e:8f
Source MAC:  c4:ca:2b:7d:5a:63
Source MAC:  a8:a1:59:f2:3e:8f
Source MAC:  c4:ca:2b:7d:5a:63
Source MAC:  c4:ca:2b:7d:5a:63
Source MAC:  a8:a1:59:f2:3e:8f
Source MAC:  a8:a1:59:f2:3e:8f
Source MAC:  c4:ca:2b:7d:5a:63
Source MAC:  00:1a:f4:f8:a5:30
Source MAC:  c4:ca:2b:7d:5a:63
Source MAC:  c4:ca:2b:7d:5a:63
Source MAC:  a8:a1:59:f2:3e:8f
Source MAC:  00:1a:f4:f8:a5:d9
Source MAC:  00:1a:f4:f8:a5:d9
Source MAC:  a8:a1:59:f2:3e:8f
Source MAC:  a8:a1:59:f2:3e:8f
Source MAC:  00:1a:f4:f8:a5:d9
Source MAC:  c4:ca:2b:7d:5a:63
Source MAC:  c4:ca:2b:7d:5a:63
Source MAC:  c4:ca:2b:7d:5a:63
Source MAC:  c4:ca:2b:7d:5a:63
Source MAC:  a8:a1:59:f2:3e:8f
Source M

In [11]:
packet = packets[0]
print(type(packet[Ether]))

<class 'scapy.layers.l2.Ether'>


### Ether 타입

In [5]:
# 패킷을 순회하며 Ethernet 2 패킷의 타입(Type) 필드 추출
for packet in packets:
    if Ether in packet:
        eth_packet = packet[Ether]
        eth_type = eth_packet.type

        # 추출된 Ethernet 2 타입 출력
        print("Ethernet 2 Type: ", hex(eth_type))

Ethernet 2 Type:  0x800
Ethernet 2 Type:  0x806
Ethernet 2 Type:  0x800
Ethernet 2 Type:  0x800
Ethernet 2 Type:  0x806
Ethernet 2 Type:  0x806
Ethernet 2 Type:  0x800
Ethernet 2 Type:  0x86dd
Ethernet 2 Type:  0x800
Ethernet 2 Type:  0x800
Ethernet 2 Type:  0x800
Ethernet 2 Type:  0x800
Ethernet 2 Type:  0x800
Ethernet 2 Type:  0x800
Ethernet 2 Type:  0x800
Ethernet 2 Type:  0x800
Ethernet 2 Type:  0x800
Ethernet 2 Type:  0x806
Ethernet 2 Type:  0x806
Ethernet 2 Type:  0x800
Ethernet 2 Type:  0x800
Ethernet 2 Type:  0x800
Ethernet 2 Type:  0x806
Ethernet 2 Type:  0x806
Ethernet 2 Type:  0x800
Ethernet 2 Type:  0x800
Ethernet 2 Type:  0x806
Ethernet 2 Type:  0x800
Ethernet 2 Type:  0x800
Ethernet 2 Type:  0x806
Ethernet 2 Type:  0x800
Ethernet 2 Type:  0x800
Ethernet 2 Type:  0x800
Ethernet 2 Type:  0x800
Ethernet 2 Type:  0x800
Ethernet 2 Type:  0x800
Ethernet 2 Type:  0x800
Ethernet 2 Type:  0x800
Ethernet 2 Type:  0x800
Ethernet 2 Type:  0x800
Ethernet 2 Type:  0x800
Ethernet 2 Type

# IP

## version

In [9]:
# 패킷을 순회하며 IP 패킷의 버전(Version) 필드 추출
for packet in packets:
    if IP in packet:
        ip_packet = packet[IP]
        ip_version = ip_packet.version

        # 추출된 IP 버전 출력 
        print("IP Version: ", ip_version)
        

IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version:  4
IP Version

In [10]:
# 패킷을 순회하며 IP 패킷의 헤더 길이(Header Length) 필드 추출
for packet in packets:
    if IP in packet:
        ip_packet = packet[IP]
        header_length = ip_packet.ihl * 4  # 헤더 길이 = IHL * 4

        # 추출된 헤더 길이 출력
        print("Header Length: ", header_length)

Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Length:  20
Header Lengt

In [11]:
# 패킷을 순회하며 IP 패킷의 전체 길이(Total Length) 필드 추출
for packet in packets:
    if IP in packet:
        ip_packet = packet[IP]
        total_length = ip_packet.len

        # 추출된 전체 길이 출력
        print("Total Length: ", total_length)

Total Length:  678
Total Length:  684
Total Length:  150
Total Length:  684
Total Length:  40
Total Length:  83
Total Length:  40
Total Length:  83
Total Length:  40
Total Length:  97
Total Length:  90
Total Length:  40
Total Length:  1022
Total Length:  90
Total Length:  83
Total Length:  40
Total Length:  77
Total Length:  118
Total Length:  40
Total Length:  97
Total Length:  90
Total Length:  40
Total Length:  83
Total Length:  202
Total Length:  97
Total Length:  40
Total Length:  83
Total Length:  97
Total Length:  40
Total Length:  83
Total Length:  664
Total Length:  90
Total Length:  90
Total Length:  40
Total Length:  203
Total Length:  90
Total Length:  790
Total Length:  315
Total Length:  97
Total Length:  83
Total Length:  40
Total Length:  90
Total Length:  90
Total Length:  40
Total Length:  90
Total Length:  90
Total Length:  40
Total Length:  203
Total Length:  90
Total Length:  90
Total Length:  40
Total Length:  90
Total Length:  90
Total Length:  40
Total Length:  

In [12]:
# 패킷을 순회하며 IP 패킷의 Time to Live (TTL) 필드 추출
for packet in packets:
    if IP in packet:
        ip_packet = packet[IP]
        ttl = ip_packet.ttl

        # 추출된 TTL 값 출력
        print("Time to Live (TTL): ", ttl)

Time to Live (TTL):  128
Time to Live (TTL):  1
Time to Live (TTL):  50
Time to Live (TTL):  1
Time to Live (TTL):  127
Time to Live (TTL):  127
Time to Live (TTL):  128
Time to Live (TTL):  127
Time to Live (TTL):  128
Time to Live (TTL):  127
Time to Live (TTL):  127
Time to Live (TTL):  128
Time to Live (TTL):  128
Time to Live (TTL):  127
Time to Live (TTL):  127
Time to Live (TTL):  128
Time to Live (TTL):  128
Time to Live (TTL):  128
Time to Live (TTL):  127
Time to Live (TTL):  127
Time to Live (TTL):  127
Time to Live (TTL):  128
Time to Live (TTL):  127
Time to Live (TTL):  1
Time to Live (TTL):  127
Time to Live (TTL):  128
Time to Live (TTL):  127
Time to Live (TTL):  127
Time to Live (TTL):  128
Time to Live (TTL):  127
Time to Live (TTL):  128
Time to Live (TTL):  127
Time to Live (TTL):  127
Time to Live (TTL):  128
Time to Live (TTL):  1
Time to Live (TTL):  127
Time to Live (TTL):  128
Time to Live (TTL):  64
Time to Live (TTL):  127
Time to Live (TTL):  127
Time to Li

In [13]:
# 패킷을 순회하며 IP 패킷의 프로토콜(Protocol) 필드 추출
for packet in packets:
    if IP in packet:
        ip_packet = packet[IP]
        protocol = ip_packet.proto

        # 추출된 프로토콜 값 출력
        print("Protocol: ", protocol)

Protocol:  6
Protocol:  17
Protocol:  6
Protocol:  17
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  17
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  17
Protocol:  6
Protocol:  6
Protocol:  17
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  17
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  17
Protocol:  17
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Protocol:  6
Prot

In [14]:
# 패킷을 순회하며 IP 패킷의 소스 IP 주소 추출
for packet in packets:
    if IP in packet:
        ip_packet = packet[IP]
        source_ip = ip_packet.src

        # 추출된 소스 IP 주소 출력
        print("Source IP Address: ", source_ip)

Source IP Address:  163.152.219.203
Source IP Address:  163.152.207.40
Source IP Address:  110.76.140.197
Source IP Address:  163.152.207.40
Source IP Address:  163.152.223.142
Source IP Address:  163.152.223.142
Source IP Address:  163.152.219.203
Source IP Address:  163.152.223.142
Source IP Address:  163.152.219.203
Source IP Address:  163.152.223.142
Source IP Address:  163.152.223.142
Source IP Address:  163.152.219.203
Source IP Address:  163.152.219.203
Source IP Address:  163.152.223.142
Source IP Address:  163.152.223.142
Source IP Address:  163.152.219.203
Source IP Address:  163.152.219.203
Source IP Address:  163.152.219.203
Source IP Address:  163.152.223.142
Source IP Address:  163.152.223.142
Source IP Address:  163.152.223.142
Source IP Address:  163.152.219.203
Source IP Address:  163.152.223.142
Source IP Address:  163.152.207.23
Source IP Address:  163.152.223.142
Source IP Address:  163.152.219.203
Source IP Address:  163.152.223.142
Source IP Address:  163.152.223.

In [15]:
# 패킷을 순회하며 IP 패킷의 목적지 IP 주소 추출
for packet in packets:
    if IP in packet:
        ip_packet = packet[IP]
        dst_ip = ip_packet.dst

        # 추출된 목적 IP 주소 출력
        print("Dest IP Address: ", dst_ip)

Dest IP Address:  163.152.223.142
Dest IP Address:  239.255.255.250
Dest IP Address:  163.152.219.203
Dest IP Address:  239.255.255.250
Dest IP Address:  163.152.219.203
Dest IP Address:  163.152.219.203
Dest IP Address:  110.76.140.197
Dest IP Address:  163.152.219.203
Dest IP Address:  163.152.223.142
Dest IP Address:  163.152.219.203
Dest IP Address:  163.152.219.203
Dest IP Address:  163.152.223.142
Dest IP Address:  163.152.223.142
Dest IP Address:  163.152.219.203
Dest IP Address:  163.152.219.203
Dest IP Address:  163.152.223.142
Dest IP Address:  163.152.223.142
Dest IP Address:  163.152.223.142
Dest IP Address:  163.152.219.203
Dest IP Address:  163.152.219.203
Dest IP Address:  163.152.219.203
Dest IP Address:  163.152.223.142
Dest IP Address:  163.152.219.203
Dest IP Address:  239.255.255.250
Dest IP Address:  163.152.219.203
Dest IP Address:  163.152.223.142
Dest IP Address:  163.152.219.203
Dest IP Address:  163.152.219.203
Dest IP Address:  163.152.223.142
Dest IP Address

In [33]:
# 패킷을 순회하며 소스 IP 주소 추출
for packet in packets:
    if IP in packet:
        ip_packet = packet[IP]
        src_ip = ip_packet.src

        # 추출된 목적 IP 주소 출력
        print("Src IP Address: ", src_ip)

Src IP Address:  163.152.219.203
Src IP Address:  163.152.207.40
Src IP Address:  110.76.140.197
Src IP Address:  163.152.207.40
Src IP Address:  163.152.223.142
Src IP Address:  163.152.223.142
Src IP Address:  163.152.219.203
Src IP Address:  163.152.223.142
Src IP Address:  163.152.219.203
Src IP Address:  163.152.223.142
Src IP Address:  163.152.223.142
Src IP Address:  163.152.219.203
Src IP Address:  163.152.219.203
Src IP Address:  163.152.223.142
Src IP Address:  163.152.223.142
Src IP Address:  163.152.219.203
Src IP Address:  163.152.219.203
Src IP Address:  163.152.219.203
Src IP Address:  163.152.223.142
Src IP Address:  163.152.223.142
Src IP Address:  163.152.223.142
Src IP Address:  163.152.219.203
Src IP Address:  163.152.223.142
Src IP Address:  163.152.207.23
Src IP Address:  163.152.223.142
Src IP Address:  163.152.219.203
Src IP Address:  163.152.223.142
Src IP Address:  163.152.223.142
Src IP Address:  163.152.219.203
Src IP Address:  163.152.223.142
Src IP Address

# TLS

In [50]:
# 패킷을 순회하if packet.haslayer(TLS):
for packet in packets:
     if TLS in packet:    
        tls_packet = packet[TLS]
               
        # TLS 레이어 필드 접근
        tls_version = tls_packet.version
        tls_length = tls_packet.len
        print(tls_packet.msg)
        print(tls_version, tls_length )

[<TLSClientHello  msgtype=client_hello msglen=163 version=TLS 1.2 gmt_unix_time=Tue, 27 Jun 2023 07:10:33 +0900 (1687849833) random_bytes=b755d480b211b058e2f35e938e5d20bf3bd012fc7c5d10479473d566 sidlen=0 sid='' cipherslen=36 ciphers=[TLS_ECDHE_ECDSA_WITH_AES_256_GCM_SHA384, TLS_ECDHE_ECDSA_WITH_AES_128_GCM_SHA256, TLS_ECDHE_RSA_WITH_AES_256_GCM_SHA384, TLS_ECDHE_RSA_WITH_AES_128_GCM_SHA256, TLS_ECDHE_ECDSA_WITH_AES_256_CBC_SHA384, TLS_ECDHE_ECDSA_WITH_AES_128_CBC_SHA256, TLS_ECDHE_RSA_WITH_AES_256_CBC_SHA384, TLS_ECDHE_RSA_WITH_AES_128_CBC_SHA256, TLS_ECDHE_ECDSA_WITH_AES_256_CBC_SHA, TLS_ECDHE_ECDSA_WITH_AES_128_CBC_SHA, TLS_ECDHE_RSA_WITH_AES_256_CBC_SHA, TLS_ECDHE_RSA_WITH_AES_128_CBC_SHA, TLS_RSA_WITH_AES_256_GCM_SHA384, TLS_RSA_WITH_AES_128_GCM_SHA256, TLS_RSA_WITH_AES_256_CBC_SHA256, TLS_RSA_WITH_AES_128_CBC_SHA256, TLS_RSA_WITH_AES_256_CBC_SHA, TLS_RSA_WITH_AES_128_CBC_SHA] complen=1 comp=null extlen=86 ext=[<TLS_Ext_ServerName  type=server_name len=21 servernameslen=19 serverna


771 1
[<TLSApplicationData  data='\x00\x00\x00\x00\x00\x00\x00\x01\\xfa\\xe7\x0fI\\x85\\xd0\\xd4g\\xb5\\xb6\\x87ǉ\\x86A\\xbf\\xa8\\xf3\\xb5i\\x80\\xae\x05\\xd1\x1en\\xe3\\x85\\xd1\\xcd\\xdaа\\xfbs\x1c\\xa0\x01\\x89\\xd8}2\\xd5r\\x8c\\xc5G\\xb8\\xb8M\x0cQ`\x0e\x06\x7f\\x8a}q\\x9b\\xd0L\\x82\\xad"7\\xa0\\xae\\x9fL\\xb3M=\\xbe\x0fL\x11o+\\x93`\\x88Zc\\xa0\\xf4UL\x01\x04\\xf9\\xcf\x0bmzd\\xfd\\xb0J\\xc1u\\xfb\\xc4uӟ\\xd6-\\xf7M\\x95i\\x94\x18\\xa9\\x82\\x96t^{\\xbe\x16\\xfdH\\xebw\\xec\\xc4L|\x18\\x83\\xbd\\xd5P\\xf9S\\x91I?\\xc3It:Gt\\xb9\\x83\x16\\x8c\\xebm\\xbc\\xff~tY\\x9b\\xec\\x9d|\\x9f;;ez\\x95klv\\xf5\t\\xf9\\x89\x1c\\xc9w\\xd0\\xe3\\xcd\\xe1\\xa4]\\xe6-\x04\\xdeD\\xb7w\\x99\x13=$\\x93|\\xd0\x15kRޓA\\x8a\\xc0aF\\xf1:\\xba\\xf2\\x94ٶ\\xac\\xafM\\xa7\\xd9\\xffT\\xba\\xd2\\xd2\x08\\xd3\\xfe\\x8d\\xecC̓}\\x88\\xaes\\xc1"\\xd6\'>\x1f\x11\\xc6\\xdaN&\\x90fz|\x00\\xd4j\\x9d\\xa2\\x8e\\xd7\\xe0\\x9b$\\xa3\\xda֝\\xf8\\xad\\x9f/\\xc27\\xc7\\xe0\\x92\\xf0t7#\\xc8J\\x8b?̅\\x8d\x05\r\\xa8\\x8a

In [74]:

for packet in packets:
    if TLS in packet:  # TLS v1.3은 version이 772입니다.
        tls_name = packet[TLS].name
        print(tls_name)
        # for extension in tls_extensions:
        #     if extension.type == 23:  # Opaque 타입의 값은 23입니다.
        #         opaque_type = extension.value
        #         print("Opaque Type:", opaque_type)

TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS
TLS


In [78]:
for packet in packets:
    if TLS in packet:  # TLS v1.3은 version이 772입니다.
        tls_name = packet[TLS].name
        print(packet.dispatch_hook())

<class 'scapy.layers.l2.Ether'>
<class 'scapy.layers.l2.Ether'>
<class 'scapy.layers.l2.Ether'>
<class 'scapy.layers.l2.Ether'>
<class 'scapy.layers.l2.Ether'>
<class 'scapy.layers.l2.Ether'>
<class 'scapy.layers.l2.Ether'>
<class 'scapy.layers.l2.Ether'>
<class 'scapy.layers.l2.Ether'>
<class 'scapy.layers.l2.Ether'>
<class 'scapy.layers.l2.Ether'>
<class 'scapy.layers.l2.Ether'>
<class 'scapy.layers.l2.Ether'>
<class 'scapy.layers.l2.Ether'>
<class 'scapy.layers.l2.Ether'>
<class 'scapy.layers.l2.Ether'>
<class 'scapy.layers.l2.Ether'>
<class 'scapy.layers.l2.Ether'>
<class 'scapy.layers.l2.Ether'>
<class 'scapy.layers.l2.Ether'>
<class 'scapy.layers.l2.Ether'>
<class 'scapy.layers.l2.Ether'>
<class 'scapy.layers.l2.Ether'>
<class 'scapy.layers.l2.Ether'>
<class 'scapy.layers.l2.Ether'>
<class 'scapy.layers.l2.Ether'>
<class 'scapy.layers.l2.Ether'>
<class 'scapy.layers.l2.Ether'>
<class 'scapy.layers.l2.Ether'>
<class 'scapy.layers.l2.Ether'>
<class 'scapy.layers.l2.Ether'>
<class '

In [94]:
for packet in packets:
    if TLS in packet:  # TLS v1.3은 version이 772입니다.
        
        print(packet[TLS].msg[0])

TypeError: __str__ returned non-string (type bytes)

In [114]:

for packet in packets:
    if TCP in packet:  # TLS v1.3은 version이 772입니다.
        if TLS in packet and (packet[TCP].dport == 443 or packet[TCP].sport == 443):
        
            if packet[TLS].haslayer(ServerName):
                sni = packet[TLS][ServerName].servername.decode()
                print(sni)

AttributeError: servernames

In [120]:
for packet in packets:
    if TCP in packet:  # TLS v1.3은 version이 772입니다.
        if TLS in packet and (packet[TCP].dport == 443 or packet[TCP].sport == 443):
        
            if packet[TLS].haslayer(ServerName):
                sni = packet[TLS][ServerName].servername.decode()
                print(sni)

ase.autodesk.com
nav-edge.smartscreen.microsoft.com
ase.autodesk.com
assets.msn.com
prd.autocad.com
developer.api.autodesk.com
ase.autodesk.com
prd.autocad.com
ase.autodesk.com
prd.autocad.com
developer.api.autodesk.com
ase.autodesk.com
dns.google
dns.google
app.autocad360.com
developer.api.autodesk.com
reporting-api.eccr.autodesk.com
ase.autodesk.com
reporting-api.eccr.autodesk.com


In [135]:
from scapy.all import *
from scapy.layers.tls.record import TLS

def extract_signature_algorithm(packet):
    if TLS in packet:
        tls_packet = packet[TLS]
        if tls_packet.type == 22:  # 핸드셰이크 메시지 타입인지 확인 (22는 핸드셰이크 메시지 타입 값)
            client_hello = tls_packet.payload
            extensions = client_hello.extensions
            for extension in extensions:
                if isinstance(extension, TLSServerName):  # 서버 이름 확장 타입 확인
                    server_name = extension.servername.decode()  # 서버 이름 추출
                    return server_name
    return None

# 패킷 읽기
packets = rdpcap('LIVE1.pcap')

# TLS 패킷에서 서버 이름 추출
server_names = []
for packet in packets:
    name = extract_server_name(packet)
    if name:
        server_names.append(name)

# 추출된 서버 이름 출력
for name in server_names:
    print(name)

NameError: name 'extract_server_name' is not defined